# Climate Data Cleaning and Transformation Notebook
This project involves analyzing and cleaning a global climate change dataset. We'll transform the dataset into a structured format suitable for analysis, perform data cleaning, and export the cleaned data.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For displaying all columns
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv("climate_change_download_0(Data).csv")  # Replace with actual filename
print("Initial data shape:", df.shape)
df.head()

Initial data shape: (13512, 28)


,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.57,..,..,..,..,..,..,..,..,..,29.57,..,..,..,..,..,..,..,..,..,..,..
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.00,..,..,..,..,..,..,..,..,..,0.00,..,..,..,..,..,..,..,..,..,..,..
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.00,..,..,..,..,..,..,..,..,..,0.00,..,..,..,..,..,..,..,..,..,..,..
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.21,..,..,..,..,..,..,..,..,..,0.21,..,..,..,..,..,..,..,..,..,..,..
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.97,..,..,..,..,..,..,..,..,..,4.97,..,..,..,..,..,..,..,..,..,..,..


In [6]:
print("\nSummary statistics:")
print(df.describe(include='all'))
print("\nMissing values per column:")
print(df.isnull().sum())


Summary statistics:
       Country code Country name     Series code  \
count         13512        13512           13512   
unique          233          233              58   
top             ABW        Aruba  AG.LND.EL5M.ZS   
freq             58           58             233   

                                Series name  SCALE Decimals   1990   1991  \
count                                 13512  13512    13512  10017  10017   
unique                                   58      2        3   3540   2860   
top     Land area below 5m (% of land area)      0        1     ..     ..   
freq                                    233  10017     5823   5163   6520   

         1992   1993   1994   1995   1996   1997   1998   1999   2000   2001  \
count   10017  10017  10017  10017  10017  10017  10017  10017  10017  10017   
unique   2981   3034   3103   3572   3105   3085   3163   3305   4026   3321   
top        ..     ..     ..     ..     ..     ..     ..     ..     ..     ..   
freq     636

## Initial Project Goals
- Understand structure of climate data
- Clean missing and inconsistent entries
- Convert data to a tidy, usable format
- Export cleaned data for future analysis

In [7]:
# 6. Data Cleaning
# 6.1 Dealing with Missing Values
missing_ratio = df.isnull().sum() / len(df)
print("\nMissing ratio per column:")
print(missing_ratio)


Missing ratio per column:
Country code    0.000000
Country name    0.000000
Series code     0.000000
Series name     0.000000
SCALE           0.000000
Decimals        0.000000
1990            0.258659
1991            0.258659
1992            0.258659
1993            0.258659
1994            0.258659
1995            0.258659
1996            0.258659
1997            0.258659
1998            0.258659
1999            0.258659
2000            0.258659
2001            0.258659
2002            0.258659
2003            0.258659
2004            0.258659
2005            0.258659
2006            0.258659
2007            0.258659
2008            0.258659
2009            0.258659
2010            0.258659
2011            0.083629
dtype: float64


In [8]:
# 6.2 Transform Columns into Numeric Types
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_numeric(df[col].str.replace(',', ''), errors='coerce')
        except:
            continue

In [9]:
# 6.3 Rename Features (if needed)
df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

In [10]:
# 6.4 Remove Empty Columns and Rows
empty_cols = df.columns[df.isnull().all()]
empty_rows = df.index[df.isnull().all(axis=1)]
df.drop(columns=empty_cols, inplace=True)
df.drop(index=empty_rows, inplace=True)
print("\nShape after removing empty columns/rows:", df.shape)


Shape after removing empty columns/rows: (10437, 24)


In [11]:
# 7. Data Frame Transformation (Optional - Based on Structure)
# Example melting format (adjust to your dataset)
# df_melted = pd.melt(df, id_vars=['country'], var_name='year', value_name='value')
# df_transformed = df_melted.pivot_table(index='country', columns='year', values='value')

In [12]:
# 8. Removal of Missing Values
print("\nRemaining missing values:")
print(df.isnull().sum())

df_cleaned = df.dropna(thresh=int(0.9 * df.shape[1]))
df_cleaned = df_cleaned.dropna(axis=1, thresh=int(0.9 * df.shape[0]))
print("\nFinal cleaned shape:", df_cleaned.shape)


Remaining missing values:
scale        420
decimals     420
1990        5583
1991        6940
1992        6784
1993        6720
1994        6658
1995        5765
1996        6633
1997        6670
1998        6619
1999        6432
2000        4941
2001        6419
2002        6380
2003        6394
2004        6212
2005        5353
2006        6201
2007        6189
2008        5834
2009        6676
2010        8105
2011        9751
dtype: int64

Final cleaned shape: (3103, 0)


In [14]:
# 9. Export Cleaned Data
df_cleaned.to_csv("cleaned_climate_data.csv", index=False)
print("\nCleaned data exported as 'cleaned_climate_data.csv'")


Cleaned data exported as 'cleaned_climate_data.csv'
